# Notebook 0
It is a notebook which shows how use DataSource, Data and EventFamily

In [2]:
import time
import pandas as pd
from IPython.core.display import display, HTML
from pprint import pprint
from datetime import datetime
from th2_data_services.data_source import DataSource
from th2_data_services.data import Data
from th2_data_services.events_tree import EventsTree
from pandas import DataFrame

# This settings for increase display jupyter notebook and dataframe table.
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_rows = 550
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

### We set up interval for th2-rpt-data-provider

In [11]:
DEMO_HOST = "th2-kube-demo" # Host port where rpt-data-provider is located.
DEMO_PORT = "30999" # Node port of rpt-data-provider.

# Create DataSource class with host and route.
# You can change host and route using propeties.
data_source = DataSource(F"http://{DEMO_HOST}:{DEMO_PORT}")

START_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=44, second=41, microsecond=692724)
END_TIME = datetime(year=2021, month=6, day=20, hour=10, minute=45, second=49, microsecond=28579)

In [12]:
# For understand which event type on based name we get from stream.
def get_super_type(record, tree):
    name = record.get("eventName")
    parent_id = record.get("parentEventId")
    super_type = record.get("eventType")
    if super_type == "":
        if "Recon" in name:
            super_type = "Recon Folder"
        else:
            if not parent_id:
                super_type = "Test Run"
            else:
                parent_event = tree.get(parent_id)
                if parent_event:
                    parent_super_type = get_super_type(parent_event, tree)
                    if parent_super_type == "Test Run":
                        super_type = "Test Case"
                    elif parent_super_type == "Recon Folder":
                        super_type = "Recon Rule"
                    elif parent_super_type == "Recon Rule":
                        super_type = "Recon Status"
                    elif parent_super_type == "Recon Status":
                        super_type = "Recon Event"

    return super_type

# Base extract (transform function)
# record is required arguments.
def extract_basic(record):
    new_object = {
            "super_type": get_super_type(record, tree),
            "status": "SUCCESSFUL" if record.get("successful") else "FAILED",
            "body": record.get("body"),
            "parentEventId": record.get("parentEventId"),
            "eventId": record.get("eventId"),
            "eventName": record.get("eventName"),
            "body": record.get("body")
        }
    return new_object

# Checking ancestor (filter function)
# record is required arguments.
def is_test_case_ancestor(record):
    ancestor = events_tree.get_ancestor_by_super_type(record, "Test Case", get_super_type)
    if ancestor:
        return True
    return False

### We build events tree for furtuher assistance

We build events_tree for further assistance:
* event_tree = EventsTree(data) 

Here we get events which doesn't exist in data source interval:
* event_tree.recover_unknown_events(data_source)

In [13]:
# You can change anything in sse_request_to_data_provider. 
# Arguments gets based on route for th2-rpt-data-provider.
# This example shows pull the events.
data = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False,
    attachedMessages=True,
    cache=True
)

events_tree = EventsTree(data)
tree = events_tree.events

events_tree.recover_unknown_events(data_source)
    
print("[INFO] Tree of parents has built!")

[INFO] Tree of parents has built!


### Demo for events.

In [14]:
working_data = data_source.get_events_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    metadataOnly=False
)

for i in working_data:
    pprint(i)

{'attachedMessageIds': [],
 'batchId': None,
 'body': {},
 'endTimestamp': {'epochSecond': 1624185888, 'nano': 167676000},
 'eventId': '84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 'eventName': "[TS_1]Aggressive IOC vs two orders: second order's price is "
              'lower than first',
 'eventType': '',
 'isBatched': False,
 'parentEventId': None,
 'startTimestamp': {'epochSecond': 1624185881, 'nano': 821639000},
 'successful': False,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': None,
 'body': {},
 'endTimestamp': {'epochSecond': 1624185888, 'nano': 169710000},
 'eventId': '88a3ee80-d1b4-11eb-b0fb-199708acc7bc',
 'eventName': 'Case[TC_1.1]: Trader DEMO-CONN1 vs trader DEMO-CONN2 for '
              'instrument INSTR1',
 'eventType': '',
 'isBatched': False,
 'parentEventId': '84db48fc-d1b4-11eb-b0fb-199708acc7bc',
 'startTimestamp': {'epochSecond': 1624185888, 'nano': 169672000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': None,
 'body': [{'

 'batchId': None,
 'body': [{'data': 'Contains comparisons', 'type': 'message'}],
 'endTimestamp': {'epochSecond': 1624185896, 'nano': 625252000},
 'eventId': '8d722b52-d1b4-11eb-9278-591e568ad66e',
 'eventName': 'Check messages',
 'eventType': 'checkMessages',
 'isBatched': False,
 'parentEventId': '8d6e0c9e-d1b4-11eb-9278-591e568ad66e',
 'startTimestamp': {'epochSecond': 1624185896, 'nano': 205141000},
 'successful': True,
 'type': 'event'}
{'attachedMessageIds': [],
 'batchId': '53c1abd6-b2f9-4378-b061-434a3315c4a2',
 'body': [{'fields': {'AccountType': {'actual': '1',
                                      'expected': '1',
                                      'key': False,
                                      'operation': 'EQUAL',
                                      'status': 'PASSED',
                                      'type': 'field'},
                      'ClOrdID': {'actual': '9601585',
                                  'expected': '9601585',
                            

                                     'type': 'field'},
                      'SecurityIDSource': {'actual': '8',
                                           'expected': 'null',
                                           'key': False,
                                           'operation': 'EQUAL',
                                           'status': 'NA',
                                           'type': 'field'},
                      'Side': {'actual': '1',
                               'expected': 'null',
                               'key': False,
                               'operation': 'EQUAL',
                               'status': 'NA',
                               'type': 'field'},
                      'Text': {'actual': 'Simulated New Order Buy is placed',
                               'expected': 'null',
                               'key': False,
                               'operation': 'EQUAL',
                               'status': 'NA',
                 

                                                                 'operation': 'EQUAL',
                                                                 'type': 'collection'}},
                                       'key': False,
                                       'operation': 'EQUAL',
                                       'type': 'collection'},
                      'TransactTime': {'actual': '2021-06-20T10:44:57',
                                       'expected': 'null',
                                       'key': False,
                                       'operation': 'EQUAL',
                                       'status': 'NA',
                                       'type': 'field'},
                      'header': {'actual': '7',
                                 'expected': '7',
                                 'fields': {'BeginString': {'actual': 'FIXT.1.1',
                                                            'expected': 'FIXT.1.1',
                           

                                'operation': 'EQUAL',
                                'status': 'PASSED',
                                'type': 'field'},
                      'SecurityID': {'actual': 'INSTR1',
                                     'expected': 'INSTR1',
                                     'key': False,
                                     'operation': 'EQUAL',
                                     'status': 'PASSED',
                                     'type': 'field'},
                      'SecurityIDSource': {'actual': '8',
                                           'expected': '8',
                                           'key': False,
                                           'operation': 'EQUAL',
                                           'status': 'PASSED',
                                           'type': 'field'},
                      'Side': {'actual': '1',
                               'expected': '1',
                               'key': False,
   

...

                    'header': {'rows': {'BeginString': {'columns': {'fieldValue': 'FIXT.1.1'},
                                                        'type': 'row'},
                                        'BodyLength': {'columns': {'fieldValue': '310'},
                                                       'type': 'row'},
                                        'MsgSeqNum': {'columns': {'fieldValue': '1308'},
                                                      'type': 'row'},
                                        'MsgType': {'columns': {'fieldValue': '8'},
                                                    'type': 'row'},
                                        'SenderCompID': {'columns': {'fieldValue': 'FGW'},
                                                         'type': 'row'},
                                        'SendingTime': {'columns': {'fieldValue': '2021-06-20T10:45:31.287'},
                                                        'type': 'row'},
                

                      'ExecID': {'actual': '2387',
                                 'expected': '*',
                                 'key': False,
                                 'operation': 'NOT_EMPTY',
                                 'status': 'PASSED',
                                 'type': 'field'},
                      'ExecType': {'actual': 'F',
                                   'expected': 'F',
                                   'key': False,
                                   'operation': 'EQUAL',
                                   'status': 'PASSED',
                                   'type': 'field'},
                      'LastPx': {'actual': '66',
                                 'expected': '66',
                                 'key': False,
                                 'operation': 'EQUAL',
                                 'status': 'PASSED',
                                 'type': 'field'},
                      'LeavesQty': {'actual': '0',
                

                                                                                  'fields': {'PartyID': {'actual': '0',
                                                                                                         'expected': 'null',
                                                                                                         'key': False,
                                                                                                         'operation': 'EQUAL',
                                                                                                         'status': 'NA',
                                                                                                         'type': 'field'},
                                                                                             'PartyIDSource': {'actual': 'P',
                                                                                                               'expected': 'null',
        

                                  'operation': 'EQUAL',
                                  'status': 'NA',
                                  'type': 'field'},
                      'OrderQty': {'actual': '100',
                                   'expected': 'null',
                                   'key': False,
                                   'operation': 'EQUAL',
                                   'status': 'NA',
                                   'type': 'field'},
                      'Price': {'actual': '64',
                                'expected': 'null',
                                'key': False,
                                'operation': 'EQUAL',
                                'status': 'NA',
                                'type': 'field'},
                      'SecurityID': {'actual': 'INSTR5',
                                     'expected': 'INSTR5',
                                     'key': False,
                                     'operation': 'EQUAL',
 

                                                                                                              'expected': 'D',
                                                                                                              'fields': {},
                                                                                                              'key': False,
                                                                                                              'operation': 'EQUAL',
                                                                                                              'status': 'PASSED',
                                                                                                              'type': 'field'},
                                                                                            'PartyRole': {'actual': '17',
                                                                                                          'expect

### Demo for filter, map and sift.

In [15]:
data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("super_type") == "Verification")\
    .sift(limit=2)

for i in data:
    pprint(i)

{'body': [{'fields': {'AccountType': {'actual': '1',
                                      'expected': '1',
                                      'key': False,
                                      'operation': 'EQUAL',
                                      'status': 'PASSED',
                                      'type': 'field'},
                      'ClOrdID': {'actual': '9601585',
                                  'expected': '9601585',
                                  'key': True,
                                  'operation': 'EQUAL',
                                  'status': 'PASSED',
                                  'type': 'field'},
                      'CumQty': {'actual': '0',
                                 'expected': '0',
                                 'key': False,
                                 'operation': 'EQUAL',
                                 'status': 'PASSED',
                                 'type': 'field'},
                      'ExecID': {'actual'

In [16]:
data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("parentEventId"))\
    .filter(is_test_case_ancestor)\
    .sift(limit=2)

for i in data:
    pprint(i)

{'body': [{'data': 'STEP1: Trader "DEMO-CONN1" sends request to create passive '
                   'Order.',
           'type': 'message'}],
 'eventId': '8bc787fe-d1b4-11eb-bae5-57b0c4472880',
 'eventName': 'placeOrderFIX demo-conn1 - STEP1: Trader "DEMO-CONN1" sends '
              'request to create passive Order.',
 'parentEventId': '88a3ee80-d1b4-11eb-b0fb-199708acc7bc',
 'status': 'SUCCESSFUL',
 'super_type': 'placeOrderFIX'}
{'body': [{'data': "Checkpoint id '8c037f50-d1b4-11eb-ba78-1981398e00bd'",
           'type': 'message'}],
 'eventId': '6e3be13f-cab7-4653-8cb9-6e74fd95ade4:8c035903-d1b4-11eb-9278-591e568ad66e',
 'eventName': 'Checkpoint',
 'parentEventId': '8bc787fe-d1b4-11eb-bae5-57b0c4472880',
 'status': 'SUCCESSFUL',
 'super_type': 'Checkpoint'}


In [17]:
def transform_output(record):
    body = record.get("body")
    if body:
        cumqty = body[0].get("fields", {}).get("CumQty")
    
    new_obj = {
        "CumQty": cumqty
    }    
    return new_obj

data = working_data \
    .map(extract_basic) \
    .filter(lambda record: record.get("super_type") == "Verification")\
    .map(transform_output)\
    .sift(limit=5, skip=5)

for i in data:
    pprint(i)

{'CumQty': {'actual': '30',
            'expected': '30',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}
{'CumQty': {'actual': '10',
            'expected': 'null',
            'key': False,
            'operation': 'EQUAL',
            'status': 'NA',
            'type': 'field'}}
{'CumQty': {'actual': '30',
            'expected': 'null',
            'key': False,
            'operation': 'EQUAL',
            'status': 'NA',
            'type': 'field'}}
{'CumQty': {'actual': '10',
            'expected': '10',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}
{'CumQty': {'actual': '40',
            'expected': '40',
            'key': True,
            'operation': 'EQUAL',
            'status': 'PASSED',
            'type': 'field'}}


### Demo for messages.

In [18]:
streams = set()
for record in events_tree.events.values():
    messages = record.get("attachedMessageIds")
    for msg in messages:
        streams.add(msg.split(":")[0])
    
working_data = data_source.get_messages_from_data_provider(
    startTimestamp=START_TIME,
    endTimestamp=END_TIME,
    stream=list(streams)
)

for i in working_data.sift(limit=5):
    pprint(i)

{'attachedEventIds': [],
 'body': {'fields': {'AccountType': {'simpleValue': '1'},
                     'ClOrdID': {'simpleValue': '9601585'},
                     'OrdType': {'simpleValue': '2'},
                     'OrderCapacity': {'simpleValue': 'A'},
                     'OrderQty': {'simpleValue': '30'},
                     'Price': {'simpleValue': '55'},
                     'SecondaryClOrdID': {'simpleValue': '11111'},
                     'SecurityID': {'simpleValue': 'INSTR1'},
                     'SecurityIDSource': {'simpleValue': '8'},
                     'Side': {'simpleValue': '1'},
                     'TradingParty': {'messageValue': {'fields': {'NoPartyIDs': {'listValue': {'values': [{'messageValue': {'fields': {'PartyID': {'simpleValue': 'DEMO-CONN1'},
                                                                                                                                       'PartyIDSource': {'simpleValue': 'D'},
                                        

                              'sequence': '1624005455622011524',
                              'subsequence': [1]},
                       'messageType': 'ExecutionReport',
                       'timestamp': '2021-06-20T10:44:57.252Z'}},
 'bodyBase64': 'OD1GSVhULjEuMQE5PTMxMAEzNT04ATM0PTEyOTIBNDk9RkdXATUyPTIwMjEwNjIwLTEwOjQ0OjU3LjI1MAE1Nj1ERU1PLUNPTk4xATExPTI2MzgzMDQBMTQ9MAExNz0yMzQ4ATIyPTgBMzc9ODY4ATM4PTEwATM5PTABNDA9MgE0ND01NgE0OD1JTlNUUjEBNTQ9MQE1OD1TaW11bGF0ZWQgTmV3IE9yZGVyIEJ1eSBpcyBwbGFjZWQBNjA9MjAyMTA2MjAtMTA6NDQ6NTcuMDAwATE1MD0wATE1MT0xMAE1Mjg9QQE1ODE9MQE0NTM9NAE0NDg9REVNTy1DT05OMQE0NDc9RAE0NTI9NzYBNDQ4PTABNDQ3PVABNDUyPTMBNDQ4PTABNDQ3PVABNDUyPTEyMgE0NDg9MwE0NDc9UAE0NTI9MTIBMTA9MDQ3AQ==',
 'direction': 'IN',
 'messageId': 'demo-conn1:first:1624005455622011524',
 'messageType': 'ExecutionReport',
 'sessionId': 'demo-conn1',
 'timestamp': {'epochSecond': 1624185897, 'nano': 252000000},
 'type': 'message'}


In [19]:
def transform_output(record):
    new_obj = {
        "Price": record.get("body", {}).\
                        get("fields", {})\
                        .get("Price", {})\
                        .get("simpleValue")
    }
    return new_obj

data = working_data \
    .filter(lambda record: record.get("messageType") == "ExecutionReport")\
    .map(transform_output)

for i in data:
    pprint(i)

{'Price': '55'}
{'Price': '55'}
{'Price': '56'}
{'Price': '56'}
{'Price': '56'}
{'Price': '56'}
{'Price': '55'}
{'Price': '55'}
{'Price': '54'}
{'Price': '54'}
{'Price': '54'}
{'Price': '54'}
{'Price': '54'}
{'Price': '54'}
{'Price': '35'}
{'Price': '35'}
{'Price': '36'}
{'Price': '36'}
{'Price': '36'}
{'Price': '35'}
{'Price': '36'}
{'Price': '34'}
{'Price': '34'}
{'Price': '34'}
{'Price': '35'}
{'Price': '34'}
{'Price': '34'}
{'Price': '34'}
{'Price': '75'}
{'Price': '75'}
{'Price': '76'}
{'Price': '76'}
{'Price': '76'}
{'Price': '75'}
{'Price': '76'}
{'Price': '74'}
{'Price': '74'}
{'Price': '75'}
{'Price': '74'}
{'Price': '74'}
{'Price': '74'}
{'Price': '74'}
{'Price': '45'}
{'Price': '45'}
{'Price': '46'}
{'Price': '46'}
{'Price': '46'}
{'Price': '44'}
{'Price': '44'}
{'Price': '46'}
{'Price': '44'}
{'Price': '45'}
{'Price': '45'}
{'Price': '44'}
{'Price': '44'}
{'Price': '44'}
{'Price': '44'}
{'Price': '65'}
{'Price': '65'}
{'Price': '66'}
{'Price': '66'}
{'Price': '66'}
{'Price'